In [33]:
import torch
from transformers import BertTokenizer, BertModel

In [35]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\masak\anaconda3\envs\test\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\masak\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
import pandas as pd
df = pd.read_csv('../resource/openstack_data2.csv', index_col=0, header=0)
msg_df = df['msg'].apply(eval)
msg_df.head()

0    [add lun# provider_location nexenta driver., f...
1    [ensure zeros written clearing volumes note o_...
2    [factorize code nfs.py glusterfs.py suppressed...
3    [fix lhn driver allow backend name configurati...
4    [sync improvements config file generator tools...
Name: msg, dtype: object

In [37]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
special_tokens_dict = {'additional_special_tokens': ['<id>','<num>']}
tokenizer.add_special_tokens(special_tokens_dict=special_tokens_dict)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

2

In [38]:
def combined_sentences(sentences):
    return '[CLS] ' + ' [SEP] '.join(sentences) + ' [SEP]'

In [141]:
from torch.utils.data import Dataset


class BertDataset(Dataset):
    def __init__(self, data_filepath, kind='train'):
        # self.params = params
        self.df = pd.read_csv(data_filepath, header=0, index_col=0)
        self.df = self.df[self.df['kinds'] == kind]
        self.df.reset_index(inplace=True, drop=True)
        self.commit = self.df['commit_id']
        self.msg_data = self.df['msg'].apply(eval)
        self.labels = self.df['label']
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        special_tokens_dict = {'additional_special_tokens': ['<id>', '<num>']}
        self.tokenizer.add_special_tokens(
            special_tokens_dict=special_tokens_dict)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        encoded_dict = self.tokenizer.encode_plus(
            self.combined_sentences(self.msg_data[idx]),
            add_special_tokens = False,
            max_length = 512,
            pad_to_max_length = True,
            return_attention_mask = True,
            return_tensors='pt',
        )
        # print(tokenizer.decode(encoded_dict['input_ids'][0]))
        input_ids = encoded_dict['input_ids'].squeeze(0)
        attention_mask = encoded_dict['attention_mask']
        label = torch.as_tensor(int(self.labels[idx]), dtype=torch.long)
        return input_ids, attention_mask, label
    
    def combined_sentences(self, sentences):
        return '[CLS] ' + ' [SEP] '.join(sentences) + ' [SEP]'

In [142]:
from torch.utils.data import random_split, DataLoader
dataset = BertDataset('../resource/openstack_data2.csv')
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 32

train_loader = DataLoader(train_dataset, shuffle=True, drop_last=True, batch_size=batch_size, pin_memory=True, num_workers=2)
valid_loader = DataLoader(val_dataset, shuffle=False, pin_memory=True, batch_size=batch_size, num_workers=2)

In [143]:
dataset[11][0].shape

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\masak\anaconda3\envs\test\lib\site-packages\transformers\tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


torch.Size([512])

In [81]:
from transformers import BertForSequenceClassification, BertConfig

import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,
    output_attentions = False, 
    output_hidden_states = False,    
)
model.resize_token_embeddings(len(tokenizer))

cpu


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30524, 768)

In [84]:
print(dataset.tokenizer.special_tokens_map)
print(dataset.tokenizer.all_special_ids)

{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<id>', '<num>']}
[100, 102, 0, 101, 103, 30522, 30523]


In [85]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af